# Load data

In [1]:
import numpy as np 
import sklearn 
import scipy.misc

In [2]:
# loading data
x_train = np.loadtxt("data/train_x.csv", delimiter=",")
y_train = np.loadtxt("data/train_y.csv", delimiter=",") 
x_test = np.loadtxt("data/test_x.csv", delimiter=",")

# x_train = x.reshape(-1, 64, 64)
# y_train = y.reshape(-1, 1) 
# x_test = x.reshape(-1, 64, 64)

## Split training data into train / valid sets

In [3]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, train_size=0.8, test_size=0.2)
data = {
    "x_train": x_train,
    "x_valid": x_valid,
    "y_train": y_train,
    "y_valid": y_valid
}

# Baseline Linear Classifier: Linear SVM

In [4]:
from sklearn import metrics
from sklearn.svm import LinearSVC

(40000, 4096)

In [6]:
def baseline_linear_svm(data):
    """
    Using out-of-the-box linear SVM to classify data
    """
    clf = LinearSVC()
    
    y_pred = clf.fit(data["x_train"], data["y_train"]).predict(data["x_valid"])
    print(y_pred)
    return metrics.f1_score(data["y_valid"], y_pred, average="macro"), y_pred
    
score, y_pred = baseline_linear_svm(data)
print(score)

[ 3.  3.  0. ...,  3.  3.  3.]
0.0342712911785
